In [1]:
import os 
import fiona
import datetime
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import geometry
import matplotlib.pyplot as plt

In [2]:
# Create filepath for shapefile
fields_fp = "C:\\Users\\ashle\\Desktop\\KGS\\data\\Agriculture-Land\\Property Lines"
os.chdir(fields_fp)
fields = gpd.read_file('Fields_Around_SD6KS.shp')

In [3]:
# Create a filepath for OpenET data and change working directory
openET_fp = "C:\\Users\\ashle\\Desktop\\KGS\\data\\Hydrometeorology\\OpenET\\"
os.chdir(openET_fp)

In [28]:
# Import ensemble file from OpenET
ensemble = pd.read_csv('ET_monthly_ensemble_FieldsAroundSD6KS_20220708.csv')
# Import DisALEXI file from OpenET
disalexi = pd.read_csv('ET_monthly_disalexi_FieldsAroundSD6KS_20220708.csv')
# Import eeMetric file from OpenET
eemetric = pd.read_csv('ET_monthly_eemetric_FieldsAroundSD6KS_20220708.csv')
# Import geeSEBAL file from OpenET
geesebal = pd.read_csv('ET_monthly_geesebal_FieldsAroundSD6KS_20220708.csv')
# Import PT-JPL file from OpenET
ptjpl = pd.read_csv('ET_monthly_ptjpl_FieldsAroundSD6KS_20220708.csv')
# Import SIMS file from OpenET
sims = pd.read_csv('ET_monthly_sims_FieldsAroundSD6KS_20220708.csv')
# Import SSEBop file from OpenET
ssebop = pd.read_csv('ET_monthly_ssebop_FieldsAroundSD6KS_20220708.csv')

In [29]:
et = [ensemble, disalexi, eemetric, geesebal, ptjpl, sims, ssebop]
lab = ['annual_ensemble', 'annual_disalexi', 'annual_eemetric', 'annual_geesebal', 'annual_ptjpl', 'annual_sims', 'annual_ssebop']

In [30]:
annual_et = {}

i = 0

for x in et:
    x.drop('acres', inplace = True, axis = 1)
    merge_x = fields.merge(x, on = 'UID', how = 'left')
    x_fields = gpd.GeoDataFrame(merge_x)
    x_fields.time = pd.to_datetime(x_fields.time).dt.strftime('%Y')
    annual_et[lab[i]] = x_fields.dissolve(by = ['UID', 'time'], aggfunc = 'sum')
    annual_et[lab[i]].drop('area', inplace = True, axis = 1)
    annual_et[lab[i]].reset_index(inplace=True)
    i += 1

In [32]:
annual_et['annual_disalexi']

,UID,time,geometry,et_min,et_max,et_mean,et_median,et_sum
0,177799.0,2016,"POLYGON ((-414585.819 1818292.835, -415155.331...",327.0,825.0,552.166,557.0,476545.795
1,177799.0,2017,"POLYGON ((-414585.819 1818292.835, -415155.331...",297.0,726.0,488.393,502.0,421504.707
2,177799.0,2018,"POLYGON ((-414585.819 1818292.835, -415155.331...",409.0,837.0,607.199,614.0,524041.407
3,177799.0,2019,"POLYGON ((-414585.819 1818292.835, -415155.331...",415.0,933.0,620.639,623.0,535639.862
4,177799.0,2020,"POLYGON ((-414585.819 1818292.835, -415155.331...",394.0,791.0,539.698,538.0,465785.989
...,...,...,...,...,...,...,...,...
67879,1559412.0,2017,"POLYGON ((-396139.798 1849396.257, -396150.239...",444.0,918.0,687.614,654.0,185399.430
67880,1559412.0,2018,"POLYGON ((-396139.798 1849396.257, -396150.239...",455.0,1045.0,832.583,847.0,224486.902
67881,1559412.0,2019,"POLYGON ((-396139.798 1849396.257, -396150.239...",388.0,1033.0,650.621,644.0,175425.255
67882,1559412.0,2020,"POLYGON ((-396139.798 1849396.257, -396150.239...",472.0,927.0,711.538,715.0,191850.040


In [ ]:
# Create a list of strings to name the columns for ame
column_names = ['UID', 
           'time', 
           'geometry', 
           'ensemble_et_mean', 
           'disalexi_et_mean', 
           'eemetric_et_mean', 
           'geesebal_et_mean', 
           'ptjpl_et_mean', 
           'ssebop_et_mean', 
           'sims_et_mean']

# Create annual monthly ET dataframe by concatting columns from different ET algorithm data 
ame = gpd.GeoDataFrame(
pd.concat(
    [annual_ensemble['UID'],
     annual_ensemble['time'],
     annual_ensemble['geometry'],
     annual_ensemble['et_mean'],
     annual_disalexi['et_mean'],
     annual_eemetric['et_mean'],
     annual_geesebal['et_mean'],
     annual_ptjpl['et_mean'],
     annual_ssebop['et_mean'],
     annual_sims['et_mean']],
axis=1,
keys=column_names)
)

ame

ame1 = pd.melt(ame, id_vars=['UID', 'time', 'geometry'], value_vars=['ensemble_et_mean', 'disalexi_et_mean', 'eemetric_et_mean', 'geesebal_et_mean', 'ptjpl_et_mean', 'ssebop_et_mean', 'sims_et_mean'])

ame1.rename(columns={"variable": "algorithm", "value": "mean_et"}, inplace=True)
ame1

ame2 = gpd.GeoDataFrame(ame1.pivot(index = ['UID', 'algorithm'], columns = 'time', values = 'mean_et'))

ame2.reset_index(inplace=True)
m = fields.merge(ame2, on = 'UID', how = 'left')
ame3 = gpd.GeoDataFrame(m)
ame3.drop('area', inplace=True, axis=1)
ame3

ame3.head(12)

years = ['2016', '2017', '2018', '2019', '2020', '2021']

search_values = ['ensemble_et_mean', 'disalexi_et_mean']
test = ame3[ame3.algorithm.str.contains('|'.join(search_values))]

test = test.iloc[0:20]
test

testname = ['{} Mean ET'.format(row) for row in ['Ensemble', 'DisALEXI']]

fig, axs = plt.subplots(nrows = 2, ncols = 6, figsize = (12, 12))

for rows in range(2):
    for cols in range(6):
        test.plot(column=years[cols], ax=axs[rows, cols], legend=True, cmap='Blues')
        axs[rows, cols].axis('off')
        if rows == 0:
            axs[rows, cols].set_title(years[cols])
fig.tight_layout()
plt.show()

axs[0,0:6]

name = ['{} Mean ET'.format(row) for row in ['Ensemble', 'DisALEXI', 'eeMetric', 'geeSEBAL', 'PT-JPL', 'SSEBop', 'SIMS']]

fig, axs = plt.subplots(nrows = 7, ncols = 6, figsize = (12, 12))

for rows in range(7):
    for cols in range(6):
        ame3.plot(column=years[cols], ax=axs[rows, cols], legend=True, cmap='Blues')
        axs[rows, cols].axis('off')
        if rows == 0:
            axs[rows, cols].set_title(years[cols])
for ax, row in zip(axs[:,0], name):
    ax.set_ylabel(row, rotation=0, size='large')
fig.tight_layout()
plt.show()